In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
kospi_data = pd.read_csv('ks11.csv')

In [ ]:
kospi_data

In [ ]:
kospi_data['Date'] = pd.to_datetime(kospi_data['Date'])
kospi_data.set_index('Date', inplace=True)

In [ ]:
# 종가 스케일러
scaler_close = MinMaxScaler(feature_range=(0, 1))
scaled_close = scaler_close.fit_transform(kospi_data[['Close']])

# 거래량 스케일러
scaler_volume = MinMaxScaler(feature_range=(0, 1))
scaled_volume = scaler_volume.fit_transform(kospi_data[['Volume']])

In [ ]:
#시계열
X = []
y = []
#10일간 데이터 사용
for i in range(len(scaled_close) - 10):
    X.append(np.concatenate((scaled_close[i:i+10], scaled_volume[i:i+10]), axis=1))
    y.append(scaled_close[i+10, 0])

X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Sequential 모델
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X_train, y_train, batch_size=64, epochs=100, validation_split=0.1)

# 모델 평가
loss = model.evaluate(X_test, y_test)

# 평가 결과
print(f'손실 값: {loss}')


In [ ]:
# 모델 저장
model.save('full_kospi_model.h5')